# Introduction and rationale

For the current design cycle, the descion was made to construct a new parachute. The reasons for this descision were as follows:

1. Legacy knowledge about how parachutes are constructed was not passed one during the COVID era, and some of it was at risk of being lost as the last people who sewed the parachute (Roman, Zach) graduated. Sewing the new parachute would serve as knowledge transfer.
2. The rocket underwent significant modifications and as a result, became much lighter during this design cycle. This meant that the parachute could be made smaller. As this eases packing, volume constraints, tangling risks, and mass, this would be a worthwhile pursuit for the recovery system.
3. The old parachute has begun to deteriorate due to the overwhelmingly large amount of testing conducted on it. If it was chosen to fly this year, it would have possibly needed significant maintenance and cleaning. 

The format of this document is a jupyter notebook in the github simulink workspace due to the necessity of carrying out calculations and also since the calculations may involve the creation of simulink models (although this is admittedly unlikely).

*Prepared by Artem Sotnikov, initial draft March 2023*



In [78]:
# Python initialization

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
from scipy.optimize import fsolve

%load_ext ipydex.displaytools
import ipydex # For better display

FT_TO_M = 0.3048 ;
LB_TO_KG = 0.453592 ;

The ipydex.displaytools extension is already loaded. To reload it, use:
  %reload_ext ipydex.displaytools


# Sizing approach 

The parachute from the previous design cycle was relatively well characterized. It had an unreefed drag area of rouhgly $14 m^2$. This value is based on experimental data collected during Truck Test 4 and analyzed during the 2022 competition cycle using the 'truck-test-rapid-analyzer' script, code for which is available on github (https://github.com/waterloo-rocketry/truck-test-rapid-analyzer). The specific analysis file package is also uploaded to the team drive at Recovery/2023/TruckTest4AnalysisRedone. It must be noted that there is uncertainty in this unreefed area due to the necessity of data filtering via hand-picked constants. This is explored to some extent in the podium presentation and can be observed directly by changing the threshold values in the script-generated excel spreadsheet. The $14 m^2$ (corresponding to the 150 ft^2 in the original units of the calculator) is actually a conservative underestimate based on the range of values given by the data analysis, with values going up to 160 - 170 for the higher windspeed cases. Therefore, this lower value can be used with some degree of reliability. 

Given that the parachute to be sewn will employ a very similar construction, the best way to size it would be a scaling of the old parachute. In the manual (Parachute Recovery Systems Design Manual, T.W. Knacke, 1992), almost all drag area calculations are ultimately tied to the 'nomial diameter' of the parachute, which is a function of the total fabric surface area. Therefore, the selected approach for sizing the parachute would be to calculate a new nominal diamater assuming the same relative drag coefficient. 

The mass that the new parachute would need to target would be 93 lbs (which is the maximum load as predicted in the cycle 2.2 load case). This is contrasted with the old parachute that was to be used on the projected 100 lb KotS rocket (which ultimately grew to 105.51 lbs), and was sized for a 113lbs SotS rocket, which was ultimately 104.2 lbs (values taken from the respective reports, available on the drive or the website). 

Sizing parachutes could be done librally to place a large factor of safety on the landing loads. In the author's opinion, such a design philosophy is appropriate for a dual-separation system, which by its nature allows large parachutes and simple parachute bays, where tangling is not a significant concern. However, in a single-bay reefing system, a larger parachute carries a significant penalty to system development and reliability. 

Due to the nature of the reefing system, only one parachute needs to be sized, with the reefed stage being fully controlable by the length of the reefing line, which is done after the construction of the parachute itself. Thus, the driving factor of the sizing is the ground impact velocity. As per the DTEG, it is recommended at lower than 30 ft/s.

As noted in many texts on descent speeds, the descent speed of the parachute is given by:

$V_{desc} = \large \sqrt{\frac{2 m}{C_d S_o \rho}}$

Written in a form that allows for easier corrections due to altitude effects, it is also stated in the manual as:
$\therefore V_{desc} = \large \sqrt{\frac{2 m}{C_d S_o \rho_o}} \cdot \frac{1}{\sqrt{\rho \, / \rho_o}{}}$

Where (as per the original source):

> $m$ is the mass of the vehicle, in lbs  
> $C_d S_o$ is the drag area, in ft^2  
> $\rho$ is the air density, (in slugs/ft^3 !)  
> $\rho_o$ is the air density at sea level (in slugs/ft^3)  

The slug to lbs conversion is given by the acceleration due to gravity, or lbs can be used and the mass must be changed to newtons (i.e. the gravitational force on the system). This makes for easier operations with metric units:

$V_{desc} = \large \sqrt{\frac{2 m g}{C_d S_o \rho}}$

This can also be trivially re-arranged to solve for CdSo:

$C_d S_o = \large\frac{2mg} {\rho V_{desc}^2} $

In [38]:
def v_desc(m, CdSo, rho):
    return math.sqrt(2*m*(9.81)/(CdSo * rho))

def cdso(m, v, rho):
    return 2 * m * (9.81) / (rho * v**2)

Air density exists as a function of altitude and can be found in many tables. Since this is a code-capable document, all desired pressures can be calculated directly using the metric earth atmosphere model provided by NASA: https://www.grc.nasa.gov/www/k-12/airplane/atmosmet.html

Which states that pressure in (in kPa) can be calculated from temperature (in $^{\circ}$C):

$p = 101.29 \cdot \large (\frac{T + 273.1}{288.08})^{5.256}$  

Where temperature varies directly with altitude as a 'lapse rate', given by $T = 15.04 - 0.00649 \cdot h$
  
And from this, density is $\rho = p \cdot (T + 273.1))^{-1}$

In last year's report, variations in pressure and temperature were considered to model the uncertainty in these values. The methodology used (which was somewhat crude but nontheless valid) revealed that temperature varied by 10 degrees, and pressure itself could vary by about 0.7 kPa due to other weather effects.

This gives an upper and lower bound for pressure at any given altitude (ignoring the recursive variation of temperature on pressure):

$\require{color} \large \rho_{\, upper} =  \small (101.29 \,\large ((\frac{(15.04 - 0.00649 \cdot h) + 273.1}{288.08})^{5.256}) \small \colorbox{yellow}{+ 0.7}) \small \cdot ((15.04 - 0.00649 \cdot h) + 273.1 \colorbox{yellow}{- 10}))^{-1}$ 

$\require{color} \large \rho_{\, lower} = \small (101.29 \,\large ((\frac{(15.04 - 0.00649 \cdot h) + 273.1}{288.08})^{5.256}) \small \colorbox{yellow}{- 0.7}) \small \cdot ((15.04 - 0.00649 \cdot h) + 273.1 \colorbox{yellow}{+ 10}))^{-1}$ 

In [51]:
def calculate_densities(h):
    T = 15.04 - 0.00649*h
    rho_u = ((101.29*(((T + 273.1)/288.08)**(5.256))) + 0.7) * (0.2869*(T + 273.1 - 10))**(-1)
    rho_l = ((101.29*(((T + 273.1)/288.08)**(5.256))) - 0.7) * (0.2869*(T + 273.1 + 10))**(-1)
    return {"rho_u" : rho_u, "rho_l" : rho_l}

Given the ground elevation at spaceport america, which is 1,401 m, the densities can be calculated:

In [52]:
rho = calculate_densities(1401) ##:

rho := {'rho_u': 1.1189855884049338, 'rho_l': 1.0246776982951848}

---

These are the desired densities, since the final descent occurs at the ground. Now executing the calculation for drag areas (in m^2):

In [54]:
cdso_l = cdso(93*LB_TO_KG, 30*FT_TO_M, rho['rho_u']) ##:
cdso_h = cdso(93*LB_TO_KG, 30*FT_TO_M, rho['rho_l']) ##:

cdso_l := 8.846070778887182

---

cdso_h := 9.660233390512623

---

For reference (as stated earlier) the old parachute had a drag area of $14m^2$, and with it the descent speed value (under the low density) would have been:

In [71]:
v_desc(93*LB_TO_KG, 14, rho['rho_l'])/FT_TO_M # in ft/s

24.920172625321268

Sizing the parachute for the lower density case and rounding for simiplicity gives a parachute with a drag area of 10. The last parachute possessed a nominal diamater of 4.84, and this is the value being scaled in the new parachute. Therefore, the new nominal diameter must be:

In [57]:
D_o_new = 4.84 * (10/14) ##:

D_o_new := 3.4571428571428573

---

The nominal diameter is given as the diameter that would be obtained if all of the fabric was laid down in a flat circle. This is a function of the canopy's geometry:

$S = (1/4) \pi D_o^2$  

$D_o = \large\sqrt{\frac{4 \cdot S}{\pi}}$ 


This means that the new surface area must be:

In [95]:
S_new = (1/4)*(math.pi)*D_o_new**2 ##:

S_new := 9.386950620654728

---

For the ellipsoidal shape that is used on the canopy (given by Nakka's website), the minor:major axis ratio is 0.707:1, due to claimed 'structural efficiency' that Nakka doesn't really elaborate on in his website. The major diameter can be referred ot as the 'constructed diameter' or $D_c$. Given the formula for the surface area of the ellipsoid:

![ellipsoid_formula.png](Figures/ellipsoid_formula.png)

In this specific case, (and with only half the ellipsoid) it would be:

$S = 2 \pi ((1/3)(D_c^{3.2} + 2 (0.707 D_c^2)^{1.6}))^{(1/1.6)}$

This is too cumbersome to re-arrange manually, so a numerical solver will be employed:

In [91]:
def SA_ellipse(dc):
    rc = dc/2
    return 2*math.pi*((1/3)*(rc**(3.2) + 2*(0.707*rc**2)**(1.6)))**(1/1.6)



In [96]:
# fsolve works for roots of zero so I just stuck the thing in a lambda function
dc_new = fsolve(lambda dc: SA_ellipse(dc) - S_new, 4.84)[0] ##:

# Sanity check:
SA_ellipse(dc_new) ##:

dc_new := 2.71341000826823

---

(SA_ellipse(dc_new)) := 9.38695062065473

---

This is the new diameter of the parachute. With this, Richard Nakka's parachute gore calculator script (shown in a link below) can be used to plot the specific gore pattens:

https://www.nakka-rocketry.net/softw.html#PARA

Aside from the diameter, the only other parameter required by the calculator is the number of gores. For simplicity and increased structural strength, this number will be retained at 16, same as the previous parachute (even though perhaps it could be reduced; but doing an analysis on it at this point is out of scope for this year's sizing due to how delayed it has been). Putting in the parameters into the calculator yields:

![parachute_params.png](Figures/parachute_params.png)

Which is the current proposed gore construction for the 2023 parachute.